In [11]:
import pandas as pd
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from huggingface_hub import login
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
from peft import LoraConfig
import os

In [2]:
load_dotenv()

True

In [3]:
access_token = os.getenv("HF_TOKEN")
login(token=access_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
torch.cuda.empty_cache()

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1080 Ti'

In [8]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"


In [9]:
# QLoRA configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # fp4 or nf4
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [12]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [16]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)  # Define the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # Where the "pad_token" is placed

# Model config
model = AutoModelForCausalLM.from_pretrained(
    model_id,  # Model that we are going to fine-tune
    quantization_config=bnb_config,  # QLoRA config defined above
    device=0,  # Where the model is trained, set device_map="auto" loads a model onto available GPUs first.
)

ImportError: Using `bitsandbytes` 4-bit quantization requires Accelerate: `pip install 'accelerate>=0.26.0'`

In [12]:
def extract_data_from_file(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    xml_text = ET.tostring(root, encoding='utf-8').decode('utf-8')

    prompt = f"Given the following text, find the control and experimental PCEs and any passivating molecules: \n\n{xml_text}"
    prompt = "what is 9 + 10? Return just a number."
    response = pipeline(prompt, max_new_tokens=100)[0]["generated_text"]
    return response

In [13]:
print(extract_data_from_file("../../data/xmls/0.xml"))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_CUDA_bmm)